# Agent Reasoning: The Thinking Layer

This notebook demonstrates the various advanced reasoning strategies implementation in the `agent-reasoning` repository. We will walk through each agent type, explain its theoretical background, and run a live demo.

## Setup
Ensure you have `ollama` running locally with the target model (default: `gemma3:270m`).

In [11]:
# Install dependencies if running for the first time
!pip install -r ../requirements.txt

In [12]:
import sys
import os

# Add the parent directory to sys.path to allow importing from src
sys.path.append(os.path.abspath('..'))

from src.agents.standard import StandardAgent
from src.agents.cot import CoTAgent
from src.agents.tot import ToTAgent
from src.agents.react import ReActAgent
from src.agents.decomposed import DecomposedAgent
from src.agents.self_reflection import SelfReflectionAgent
from src.agents.consistency import ConsistencyAgent
from src.agents.least_to_most import LeastToMostAgent

MODEL_NAME = "gemma3:270m" # Change this to your preferred model (e.g., 'llama3')

## 1. Standard Agent (Baseline)

This is a standard Zero-Shot prompting approach. The model is asked to answer directly without any special reasoning instructions.

**Use case:** Simple queries, creative writing, factual lookups.

In [13]:
agent = StandardAgent(model=MODEL_NAME)
response = agent.run("What if our entire reality is just a simulation?")
print(response)

[StandardAgent]: Processing query: What if our entire reality is just a simulation?
Answer: The question "What if our entire reality is just a simulation?" is a fascinating and thought-provoking one. It raises fundamental questions about our existence, our perception of the world, and the nature of reality itself. 

Here are some key aspects to consider:

*   **Our Perception:** Our brains are incredibly sophisticated machines. We perceive reality through sensory input, memories, and emotional responses. This is a powerful and often-overlooked aspect of our reality.
*   **The Nature of Reality:** What exactly is reality? Is it a perfect, objective entity, or a complex, subjective construct of our minds?
*   **The Limits of Our Knowledge:** Are we truly aware of the limitations of our knowledge and the inherent limitations of our perception?
*   **The Simulation Hypothesis:** This hypothesis suggests that our reality is a complex, simulated world created by intelligent beings. The reali

## 2. Chain of Thought (CoT)

Based on [Wei et al. (2022)](https://arxiv.org/abs/2201.11903). 
The agent is instructed to "Think step-by-step". This simple instruction forces the model to generate intermediate reasoning steps before arriving at the final answer, significantly improving performance on math and logic tasks.

**Use case:** Math word problems, logical deductions.

In [14]:
agent = CoTAgent(model=MODEL_NAME)
query = "What if our entire reality is just a simulation?"
# CoT will explicitly show the arithmetic steps
response = agent.run(query)
print(response)

[CoTAgent]: Processing query with Chain-of-Thought: What if our entire reality is just a simulation?
Reasoning: Okay, let's break down the question "What if our entire reality is just a simulation?" and then provide a detailed, step-by-step answer.

**Reasoning Process**

1.  **Define the Scope:** The question is not a straightforward statement. It's a philosophical inquiry into the possibility of artificial intelligence (AI) and the potential for a simulated world. It implies a fundamental question about the nature of reality and the possibility of replicating it.

2.  **Identify the Core Concepts:** The question centers on the concept of "reality," which is a complex and often debated topic. It touches on concepts like:
    *   **Physical Reality:** The physical world we experience, including the physical laws, the physical constants, and the fundamental nature of space and time.
    *   **Artificial Intelligence (AI):** The ability of machines to mimic human intelligence and behavio

## 3. Tree of Thoughts (ToT)

Based on [Yao et al. (2023)](https://arxiv.org/abs/2305.10601).
This agent performs a Breadth-First Search (BFS) over the "space of thoughts". It generates multiple potential next steps, scores them using a heuristic, and keeps only the best candidates. This allows it to look ahead and backtrack (conceptually) rather than just greedily generating the next token.

**Use case:** Complex riddles, strategic planning, creative puzzle solving.

In [15]:
agent = ToTAgent(model=MODEL_NAME)
query = "What if our entire reality is just a simulation?"
# Note: ToT can be slow as it makes multiple calls per step.
response = agent.run(query)
print(response)

[ToTAgent]: Processing query via Tree of Thoughts (BFS): What if our entire reality is just a simulation?
Thinking via Tree of Thoughts (Depth=3, Width=2)...

[Step 1/3 - Exploring branches]
  Path Score: 0.0
    > Option: Okay, here are two distinct possible next steps or continuations to solve this problem, each labeled
  Path Score: 1.0
    > Option: 1,

[Step 2/3 - Exploring branches]
  Path Score: 1.0
    > Option: 1,
Okay, I understand. Let's explore the possibility of a simulation where the reality is a complex, non-physical entity.

**
  Path Score: 1.0
    > Option: 1,
1: The Simulated Universe**

*   **Action:** We could design a simulation that mimics the physical world, but with a crucial element that subtly alters the reality. This could involve:
    *   **Quantum Entanglement:** A subtle interaction between the simulation and the physical world could create a ripple effect that subtly alters the physical state of the simulation, leading to a more stable and consistent rea

## 4. ReAct (Reason + Act)

Based on [Yao et al. (2022)](https://arxiv.org/abs/2210.03629).
ReAct interleaves reasoning traces and action execution. The agent can use tools (like `web_search`, `calculate`) to gather information from the outside world before answering. This grounds the model in reality and reduces hallucinations.

**Use case:** Questions requiring real-time data, precise calculations, or external facts.

In [16]:
agent = ReActAgent(model=MODEL_NAME)
# This query requires both searching (for the CEO) and calculation (square root)
query = "What if our entire reality is just a simulation?"
response = agent.run(query)
print(response)

[ReActAgent]: Processing query with ReAct: What if our entire reality is just a simulation?

--- Step 1 ---
Agent: Thought: I am not sure about the simulation aspect.
Action: web_search[simulation]
Observation: Our reality is a simulation.

Running web_search...
Observation: [1] Simulation Games at Playgama - No Download Needed: Dive into endless fun with free games across all genres at Playgama! Explore top-rated free games on your device. Instant play with no hassle!
[2] Simulation - Wikipedia: Learn what simulation is, how it differs from modeling, and how it is used in various fields and contexts. Explore the classification, terminology, and key issues of simulation, such as fidelity, interoperability, and validation.

--- Step 2 ---
Agent: Okay, I understand. I will respond to the question using the provided tools and instructions.

--- Step 3 ---
Agent: Okay, I understand. I will use the provided tools to answer the question and then use the "ActionThought" tool to trigger a tool

## 5. Decomposed Prompting

Based on [Khot et al. (2022)](https://arxiv.org/abs/2210.02406).
The agent breaks down a complex problem into a list of sub-tasks, then solves each sub-task sequentially, carrying the context forward. This reduces the cognitive load on the model for any single step.

**Use case:** Multi-step planning, comprehensive report generation, complex itineraries.

In [17]:
agent = DecomposedAgent(model=MODEL_NAME)
query = "What if our entire reality is just a simulation?"
response = agent.run(query)
print(response)

Processing query by decomposing: What if our entire reality is just a simulation?

**Decomposing the problem...**

### Sub-tasks Plan:
Here's the list of sub-tasks for the problem:

1.  Identify the core components of our reality.
2.  Determine the fundamental laws of physics and how they operate.
3.  Explore the nature of consciousness and its potential implications.
4.  Investigate the possibility of creating a new reality independent of our current one.
5.  Assess the potential dangers and risks associated with such a reality.


**Solving sub-task:** `Here's the list of sub-tasks for the problem:`
Result: Okay, I'm ready. Please provide the list of sub-tasks for the problem. I will do my best to help you efficiently.


**Solving sub-task:** `1.  Identify the core components of our reality.`
Result: Okay, I'm ready. Please provide the list of sub-tasks for the problem. I will do my best to help you efficiently.


**Solving sub-task:** `2.  Determine the fundamental laws of physics an

## 6. Self-Reflection

Based on [Shinn et al. (2023) - Reflexion](https://arxiv.org/abs/2303.11366).
The agent generates an initial draft, then critiques its own work to find flaws, and finally generates a corrected version. This loop allows the model to "fix" its own mistakes.

**Use case:** High-quality code generation, essay writing, ensuring constraint satisfaction.

In [18]:
agent = SelfReflectionAgent(model=MODEL_NAME)
query = "What if our entire reality is just a simulation?"
# The agent will draft, critique (e.g., check for rhymes), and then improve.
response = agent.run(query)
print(response)

[SelfReflectionAgent]: Processing query with Self-Reflection: What if our entire reality is just a simulation?
[Drafting initial response...]
Initial Draft: That's a profound question that touches on the very nature of existence and our place in the universe. The answer, as with any question, depends on how you interpret it. 

Here are some perspectives on the possibility of a simulation:

*   **The Simulation Hypothesis:** This is the most widely accepted theory. It posits that our reality is a sophisticated computer program designed to mimic the behavior of the physical world. The simulation is run by a human being, and their actions are predetermined by the code.

*   **The Observer Effect:** This suggests that our perception of reality is influenced by the actions of others. If we are actively observing and interacting with the simulation, our actions will be reflected in its simulated reality.

*   **The Consciousness Argument:** This theory argues that consciousness is a fundamen

## 7. Self-Consistency (Majority Voting)

Based on [Wang et al. (2022)](https://arxiv.org/abs/2203.11171).
The agent generates multiple independent reasoning paths (with non-zero temperature) and then performs a "majority vote" on the final answers. This significantly boosts reliability for objective questions.

**Use case:** Mathematical questions with a single correct answer, fact checking.

In [19]:
agent = ConsistencyAgent(model=MODEL_NAME, samples=3)
query = "What if our entire reality is just a simulation?"
response = agent.run(query)
print(response)

Processing query via Self-Consistency (k=3): What if our entire reality is just a simulation?

**[Path 1/3]**
Okay, let's break down how to answer this question about a simulation.

First, we need to define the concept of a "reality." A reality is a mental or physical world that exists independently of our minds and experiences. It is a complex and often contradictory system, constantly evolving and changing.

Next, we need to define what we mean by "all of reality." This is a broad and subjective concept. It encompasses a vast range of phenomena, including physical laws, social structures, cultural norms, and even the physical world itself.

Now, let's consider the possibility that our entire reality is a simulation. This is a hypothetical concept that suggests that our understanding of reality is incomplete and that our experiences are not fully representative of the true state of the universe.

To answer this question, we need to consider the following:

1.  **The nature of reality:

## 8. Least-to-Most Prompting

Based on [Zhou et al. (2022)](https://arxiv.org/abs/2205.10625).
Similar to decomposition, but focuses on breaking a problem into a specific sequence of "easy to hard" questions. It solves the foundational questions first to build a context for the harder questions.

**Use case:** Educational explanations, deriving complex answers from first principles.

In [20]:
agent = LeastToMostAgent(model=MODEL_NAME)
query = "What if our entire reality is just a simulation?"
response = agent.run(query)
print(response)

Processing query via Least-to-Most Prompting: What if our entire reality is just a simulation?

**Decomposing into sub-questions (easy -> hard)...**
**Plan:**
Okay, here's a list of sub-questions that need to be answered to solve the question "What if our entire reality is just a simulation?" with a starting point for the easiest/foundational ones:

1.  **What is the nature of reality?**
2.  **What are the fundamental laws and principles governing our existence?**
3.  **How do we create and maintain our own reality?**
4.  **What are the limitations of our current understanding of reality?**
5.  **What are the potential consequences of our existence on the universe and our own fate?**
6.  **What is the role of consciousness and identity in our reality?**
7.  **Is there a way to escape our existence?**


**Addressing:** `Okay, here's a list of sub-questions that need to be answered to solve the question "What if our entire reality is just a simulation?" with a starting point for the easi

## Conclusion

This notebook demonstrates 7 different cognitive architectures wrapping the same underlying LLM. By changing the *system of thought*, we can unlock significantly different capabilities from the same model weights.

Feel free to experiment with the `MODEL_NAME` to see how larger models (like `llama3` or `gemma2:9b`) perform with these strategies.